In [1]:
Project = '/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte'

import pandas as pd

In [2]:
prefix = 'adi_'
Conds = ['D0','D4','D8'] 
reps = ['a','b','c']

SPs = [Cond+rep for Cond in Conds for rep in reps]

In [3]:
# Select genes have at least N (cutoff) reads for all samples
cutoff = 200

genelist = pd.DataFrame()
for SP in SPs:
    data = pd.read_excel(f'/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte/Results/Tables/Codon_analysis/codon_prediction/v20220913/{prefix}A-codon-Pred.xlsx', sheet_name=SP)
    data = data.set_index('transcript_id', drop=True)
    data = data[ data.sum(axis=1) > cutoff ]
    
    print(SP+' '+str(len(data)))
    
    if SP == 'D0a' : 
        genelist['transcript_id'] = data.reset_index()['transcript_id']
    else :
        genelist = pd.merge(genelist, data.reset_index()[['transcript_id']], on='transcript_id')
            
print('Gene number after filtering : '+str(len(genelist)))

D0a 8406
D0b 8108
D0c 8423
D4a 9037
D4b 8919
D4c 9107
D8a 8840
D8b 8911
D8c 9082
Gene number after filtering : 7215


In [4]:
for site in ['P','A']:
    result = pd.DataFrame()
    result['codon'] = [a+b+c for a in ['U','C','A','G'] for b in ['U','C','A','G'] for c in ['U','C','A','G']]
    for SP in SPs[3:]:
        ctl_data = pd.read_excel(f'/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte/Results/Tables/Codon_analysis/codon_prediction/v20220913/{prefix}{site}-codon-Pred.xlsx', sheet_name='D0'+SP[2])
        ctl_data = pd.merge(genelist, ctl_data, on='transcript_id').set_index('transcript_id', drop=True).T
        
        ctl_data2 = ctl_data.copy()
        for ID in ctl_data:
            ctl_data2[ID] =  ctl_data[ID] / ctl_data[ID].sum()
        ctl_data2 = ctl_data2.T

        data = pd.read_excel(f'/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte/Results/Tables/Codon_analysis/codon_prediction/v20220913/{prefix}{site}-codon-Pred.xlsx', sheet_name=SP)        
        data = pd.merge(genelist, data, on='transcript_id').set_index('transcript_id', drop=True).T
        
        data2 = data.copy()
        for ID in data:
            data2[ID] =  data[ID] / data[ID].sum()
        data2 = data2.T
        
        FC = (data2 - ctl_data2)*100
        
        means = []
        for codon in FC:
            means.append( FC[codon].mean() )
        
        result[SP] = means
        print(f'{SP} {site} done')
            
    result.to_csv(f'Subsequence abundance shift_{site}-site.tsv', sep='\t', index=False)

D4a P done
D4b P done
D4c P done
D8a P done
D8b P done
D8c P done
D4a A done
D4b A done
D4c A done
D8a A done
D8b A done
D8c A done
